# Hybrid search with re-ranker

## Setup AI Search and OpenAI clients

In [4]:
import os

from azure.identity import AzureDeveloperCliCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from dotenv import load_dotenv
from openai import OpenAI

from render_table import render_product_results

load_dotenv()

azure_credential = AzureDeveloperCliCredential(tenant_id=os.environ["AZURE_TENANT_ID"])
token_provider = get_bearer_token_provider(
    azure_credential, "https://cognitiveservices.azure.com/.default"
)

openai_client = OpenAI(
    base_url=f"https://{os.environ['AZURE_OPENAI_SERVICE']}.openai.azure.com/openai/v1", api_key=token_provider
)

def get_embedding_vector(query: str) -> list[float]:
    response = openai_client.embeddings.create(
        model=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"],
        input=[query],
    )
    embedding_vector = response.data[0].embedding
    return embedding_vector

search_client = SearchClient(
    f"https://{os.environ['AZURE_SEARCH_SERVICE']}.search.windows.net",
    "zava-products-index",
    credential=azure_credential,
)

## Perform hybrid search with semantic reranker

In [6]:
search_query = "100 foot hose that won't break"
search_vector = get_embedding_vector(search_query)

results = search_client.search(
    search_query,
    top=5,
    vector_queries=[VectorizedQuery(vector=search_vector, k_nearest_neighbors=50, fields="embedding")],
    query_type="semantic",
    semantic_configuration_name="products-semantic-config",
)

render_product_results(results, show_reranker=True)

┏━━━━━┳━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━┳━━━━━━━━━┓
┃ Sc… ┃ R… ┃ Name                 ┃ Description                                  ┃ Categories      ┃ P… ┃ SKU     ┃
┡━━━━━╇━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━╇━━━━━━━━━┩
│ 0.… │ 2… │ Expandable Hose      │ Lightweight expandable hose that extends     │ GARDEN &        │ $… │ GOHOS0… │
│     │    │ 100-foot             │ when water flows and contracts when emp...   │ OUTDOOR, HOSES  │    │         │
├─────┼────┼──────────────────────┼──────────────────────────────────────────────┼─────────────────┼────┼─────────┤
│ 0.… │ 2… │ Outdoor Spigot       │ Frost-free outdoor faucet with anti-siphon   │ PLUMBING,       │ $… │ PLFCT0… │
│     │    │ Faucet               │ protection for yard connections.             │ FAUCETS         │    │         │
├─────┼────┼──────────────────────┼──────────────────────────────────────────────┼─────────────────┼────┼─────────┤
│ 0.… │ 1… │ Drinking Water Safe  │ Lead-free garden hose safe for drinking      │ GARDEN &        │ $… │ GOHOS0… │
│     │    │ Hose                 │ water and filling pools.                     │ OUTDOOR, HOSES  │    │         │
├─────┼────┼──────────────────────┼──────────────────────────────────────────────┼─────────────────┼────┼─────────┤
│ 0.… │ 1… │ Retractable Hose     │ Wall-mounted hose reel that automatically    │ STORAGE &       │ $… │ SOHH00… │
│     │    │ Hanger               │ retracts garden hoses.                       │ ORGANIZATION,   │    │         │
│     │    │                      │                                              │ HOOKS & HANGERS │    │         │
├─────┼────┼──────────────────────┼──────────────────────────────────────────────┼─────────────────┼────┼─────────┤
│ 0.… │ 1… │ Heavy Duty Teflon    │ Thick PTFE tape for high-pressure and gas    │ PLUMBING,       │ $… │ PLTFL0… │
│     │    │ Tape                 │ line applications.                           │ TEFLON TAPE     │    │         │
└─────┴────┴──────────────────────┴──────────────────────────────────────────────┴─────────────────┴────┴─────────┘